In [13]:
import pandas as pd
import pulp
from pulp import *

In [5]:
ds=pd.read_csv('diet.csv')
ds.head()

Foods Serving Size  Calories  Cholesterol (mg)  \
0      Frozen Broccoli    10 Oz Pkg      73.8               0.0   
1          Frozen Corn      1/2 Cup      72.2               0.0   
2  Raw Lettuce Iceberg       1 Leaf       2.6               0.0   
3       Baked Potatoes      1/2 Cup     171.5               0.0   
4                 Tofu    1/4 block      88.2               0.0   

   Total_Fat (g)  Sodium (mg)  Carbohydrates (g)  Dietary_Fiber (g)  \
0            0.8         68.2               13.6                8.5   
1            0.6          2.5               17.1                2.0   
2            0.0          1.8                0.4                0.3   
3            0.2         15.2               39.9                3.2   
4            5.5          8.1                2.2                1.4   

   Protein (g)  Vit_A (IU)  Vit_C (IU)  Calcium (mg)  Iron (mg)  \
0          8.0      5867.4       160.2         159.0        2.3   
1          2.5       106.6         5.2           3.3        0.3   
2          0.2        66.0         0.8           3.8        0.1   
3          3.7         0.0        15.6          22.7        4.3   
4          9.4        98.6         0.1         121.8        6.2   

   Price/Serving ($)  
0               0.48  
1               0.54  
2               0.06  
3               0.18  
4               0.93

In [7]:
ds.describe()

Calories  Cholesterol (mg)  Total_Fat (g)  Sodium (mg)  \
count   17.000000         17.000000      17.000000    17.000000   
mean   110.841176         23.629412       4.029412   165.135294   
std     86.862780         57.854286       4.345078   300.034150   
min      2.600000          0.000000       0.000000     0.000000   
25%     67.200000          0.000000       0.600000     8.100000   
50%     81.000000          0.000000       3.100000    68.900000   
75%    104.900000          5.100000       5.500000   134.500000   
max    358.200000        211.200000      12.800000  1237.100000   

       Carbohydrates (g)  Dietary_Fiber (g)  Protein (g)   Vit_A (IU)  \
count          17.000000          17.000000    17.000000    17.000000   
mean           13.947059           2.170588     5.805882   587.394118   
std            15.708561           3.235499     9.844318  1543.523543   
min             0.000000           0.000000     0.200000     0.000000   
25%             1.300000           0.000000     1.100000     0.000000   
50%            11.800000           1.100000     2.500000    73.100000   
75%            17.100000           2.700000     6.700000   101.800000   
max            58.300000          11.600000    42.200000  5867.400000   

       Vit_C (IU)  Calcium (mg)  Iron (mg)  Price/Serving ($)  
count   17.000000     17.000000  17.000000          17.000000  
mean    14.117647     32.800000   1.300000           0.645882  
std     38.416553     45.261587   1.674813           0.716406  
min      0.000000      3.100000   0.100000           0.060000  
25%      0.000000      6.700000   0.400000           0.180000  
50%      0.800000     10.800000   0.600000           0.450000  
75%     10.400000     26.200000   1.800000           0.720000  
max    160.200000    159.000000   6.200000           2.520000

In [8]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Foods              17 non-null     object 
 1   Serving Size       17 non-null     object 
 2   Calories           17 non-null     float64
 3   Cholesterol (mg)   17 non-null     float64
 4   Total_Fat (g)      17 non-null     float64
 5   Sodium (mg)        17 non-null     float64
 6   Carbohydrates (g)  17 non-null     float64
 7   Dietary_Fiber (g)  17 non-null     float64
 8   Protein (g)        17 non-null     float64
 9   Vit_A (IU)         17 non-null     float64
 10  Vit_C (IU)         17 non-null     float64
 11  Calcium (mg)       17 non-null     float64
 12  Iron (mg)          17 non-null     float64
 13  Price/Serving ($)  17 non-null     float64
dtypes: float64(12), object(2)
memory usage: 2.0+ KB


In [9]:
ds.columns


Index(['Foods', 'Serving Size', 'Calories', 'Cholesterol (mg)',
       'Total_Fat (g)', 'Sodium (mg)', 'Carbohydrates (g)',
       'Dietary_Fiber (g)', 'Protein (g)', 'Vit_A (IU)', 'Vit_C (IU)',
       'Calcium (mg)', 'Iron (mg)', 'Price/Serving ($)'],
      dtype='object')

In [10]:
ds.describe(include='O')

Foods Serving Size
count                17           17
unique               17           13
top     Frozen Broccoli      1/2 Cup
freq                  1            2

In [14]:
prob=LpProblem('Diet_Problem', LpMinimize)

In [15]:
#List of food items
food = list(ds.Foods)

#The list of items
count=pd.Series(range(1,len(food)+1))
print('List of different food items is here follows: -')
food_s = pd.Series(food)

#Convert to data frame
f_frame = pd.concat([count,food_s],axis=1,keys=['S.No','Food Items'])
f_frame

List of different food items is here follows: -


S.No              Food Items
0      1         Frozen Broccoli
1      2             Frozen Corn
2      3     Raw Lettuce Iceberg
3      4          Baked Potatoes
4      5                    Tofu
5      6         Roasted Chicken
6      7      Spaghetti W/ Sauce
7      8               Raw Apple
8      9                  Banana
9     10             Wheat Bread
10    11             White Bread
11    12         Oatmeal Cookies
12    13               Apple Pie
13    14          Scrambled Eggs
14    15          Turkey Bologna
15    16        Beef Frankfurter
16    17  Chocolate Chip Cookies

In [18]:
# Create a dictinary of costs for all food items
costs = dict(zip(food,ds['Price/Serving ($)']))

#Create a dictionary of calories for all items of food
calories = dict(zip(food,ds['Calories']))

#Create a dictionary of cholesterol for all items of food
chol = dict(zip(food,ds['Cholesterol (mg)']))

#Create a dictionary of total fat for all items of food
fat = dict(zip(food,ds['Total_Fat (g)']))

#Create a dictionary of sodium for all items of food
sodium = dict(zip(food,ds['Sodium (mg)']))

#Create a dictionary of carbohydrates for all items of food
carbs = dict(zip(food,ds['Carbohydrates (g)']))

#Create a dictionary of dietary fiber for all items of food
fiber = dict(zip(food,ds['Dietary_Fiber (g)']))

#Create a dictionary of protein for all food items
protein = dict(zip(food,ds['Protein (g)']))

#Create a dictionary of vitamin A for all food items
vit_A = dict(zip(food,ds['Vit_A (IU)']))

#Create a dictionary of vitamin C for all food items
vit_C = dict(zip(food,ds['Vit_C (IU)']))

#Create a dictionary of calcium for all food items
calcium = dict(zip(food,ds['Calcium (mg)']))

#Create a dictionary of iron for all food items
iron = dict(zip(food,ds['Iron (mg)']))

In [20]:
iron

{'Frozen Broccoli': 2.3,
 'Frozen Corn': 0.3,
 'Raw Lettuce Iceberg': 0.1,
 ' Baked Potatoes': 4.3,
 'Tofu': 6.2,
 'Roasted Chicken': 1.8,
 'Spaghetti W/ Sauce': 2.3,
 'Raw Apple': 0.2,
 'Banana': 0.4,
 'Wheat Bread': 0.7,
 'White Bread': 0.8,
 'Oatmeal Cookies': 0.5,
 'Apple Pie': 0.1,
 'Scrambled Eggs': 0.7,
 'Turkey Bologna': 0.4,
 'Beef Frankfurter': 0.6,
 'Chocolate Chip Cookies': 0.4}

In [22]:
food_vars = LpVariable.dicts("Food",food,lowBound=0,cat='Continuous')
food_vars

{'Frozen Broccoli': Food_Frozen_Broccoli,
 'Frozen Corn': Food_Frozen_Corn,
 'Raw Lettuce Iceberg': Food_Raw_Lettuce_Iceberg,
 ' Baked Potatoes': Food__Baked_Potatoes,
 'Tofu': Food_Tofu,
 'Roasted Chicken': Food_Roasted_Chicken,
 'Spaghetti W/ Sauce': Food_Spaghetti_W__Sauce,
 'Raw Apple': Food_Raw_Apple,
 'Banana': Food_Banana,
 'Wheat Bread': Food_Wheat_Bread,
 'White Bread': Food_White_Bread,
 'Oatmeal Cookies': Food_Oatmeal_Cookies,
 'Apple Pie': Food_Apple_Pie,
 'Scrambled Eggs': Food_Scrambled_Eggs,
 'Turkey Bologna': Food_Turkey_Bologna,
 'Beef Frankfurter': Food_Beef_Frankfurter,
 'Chocolate Chip Cookies': Food_Chocolate_Chip_Cookies}

In [23]:
prob += lpSum([costs[i]*food_vars[i] for i in food])
prob

Diet_Problem:
MINIMIZE
0.48*Food_Apple_Pie + 0.45*Food_Banana + 0.81*Food_Beef_Frankfurter + 0.09*Food_Chocolate_Chip_Cookies + 0.48*Food_Frozen_Broccoli + 0.54*Food_Frozen_Corn + 0.27*Food_Oatmeal_Cookies + 0.72*Food_Raw_Apple + 0.06*Food_Raw_Lettuce_Iceberg + 2.52*Food_Roasted_Chicken + 0.33*Food_Scrambled_Eggs + 2.34*Food_Spaghetti_W__Sauce + 0.93*Food_Tofu + 0.45*Food_Turkey_Bologna + 0.15*Food_Wheat_Bread + 0.18*Food_White_Bread + 0.18*Food__Baked_Potatoes + 0.0
VARIABLES
Food_Apple_Pie Continuous
Food_Banana Continuous
Food_Beef_Frankfurter Continuous
Food_Chocolate_Chip_Cookies Continuous
Food_Frozen_Broccoli Continuous
Food_Frozen_Corn Continuous
Food_Oatmeal_Cookies Continuous
Food_Raw_Apple Continuous
Food_Raw_Lettuce_Iceberg Continuous
Food_Roasted_Chicken Continuous
Food_Scrambled_Eggs Continuous
Food_Spaghetti_W__Sauce Continuous
Food_Tofu Continuous
Food_Turkey_Bologna Continuous
Food_Wheat_Bread Continuous
Food_White_Bread Continuous
Food__Baked_Potatoes Continuous

In [24]:
lpSum([food_vars[i]*calories[i] for i in food])

67.2*Food_Apple_Pie + 104.9*Food_Banana + 141.8*Food_Beef_Frankfurter + 78.1*Food_Chocolate_Chip_Cookies + 73.8*Food_Frozen_Broccoli + 72.2*Food_Frozen_Corn + 81.0*Food_Oatmeal_Cookies + 81.4*Food_Raw_Apple + 2.6*Food_Raw_Lettuce_Iceberg + 277.4*Food_Roasted_Chicken + 99.6*Food_Scrambled_Eggs + 358.2*Food_Spaghetti_W__Sauce + 88.2*Food_Tofu + 56.4*Food_Turkey_Bologna + 65.0*Food_Wheat_Bread + 65.0*Food_White_Bread + 171.5*Food__Baked_Potatoes + 0.0

In [26]:
ds[['Foods','Calories']]

Foods  Calories
0          Frozen Broccoli      73.8
1              Frozen Corn      72.2
2      Raw Lettuce Iceberg       2.6
3           Baked Potatoes     171.5
4                     Tofu      88.2
5          Roasted Chicken     277.4
6       Spaghetti W/ Sauce     358.2
7                Raw Apple      81.4
8                   Banana     104.9
9              Wheat Bread      65.0
10             White Bread      65.0
11         Oatmeal Cookies      81.0
12               Apple Pie      67.2
13          Scrambled Eggs      99.6
14          Turkey Bologna      56.4
15        Beef Frankfurter     141.8
16  Chocolate Chip Cookies      78.1

In [27]:
prob += lpSum([food_vars[x]*calories[x] for x in food]) >= 800, "CaloriesMinimum"
prob += lpSum([food_vars[x]*calories[x] for x in food]) <= 1300, "CaloriesMaximum"
prob

Diet_Problem:
MINIMIZE
0.48*Food_Apple_Pie + 0.45*Food_Banana + 0.81*Food_Beef_Frankfurter + 0.09*Food_Chocolate_Chip_Cookies + 0.48*Food_Frozen_Broccoli + 0.54*Food_Frozen_Corn + 0.27*Food_Oatmeal_Cookies + 0.72*Food_Raw_Apple + 0.06*Food_Raw_Lettuce_Iceberg + 2.52*Food_Roasted_Chicken + 0.33*Food_Scrambled_Eggs + 2.34*Food_Spaghetti_W__Sauce + 0.93*Food_Tofu + 0.45*Food_Turkey_Bologna + 0.15*Food_Wheat_Bread + 0.18*Food_White_Bread + 0.18*Food__Baked_Potatoes + 0.0
SUBJECT TO
CaloriesMinimum: 67.2 Food_Apple_Pie + 104.9 Food_Banana
 + 141.8 Food_Beef_Frankfurter + 78.1 Food_Chocolate_Chip_Cookies
 + 73.8 Food_Frozen_Broccoli + 72.2 Food_Frozen_Corn + 81 Food_Oatmeal_Cookies
 + 81.4 Food_Raw_Apple + 2.6 Food_Raw_Lettuce_Iceberg
 + 277.4 Food_Roasted_Chicken + 99.6 Food_Scrambled_Eggs
 + 358.2 Food_Spaghetti_W__Sauce + 88.2 Food_Tofu + 56.4 Food_Turkey_Bologna
 + 65 Food_Wheat_Bread + 65 Food_White_Bread + 171.5 Food__Baked_Potatoes
 >= 800

CaloriesMaximum: 67.2 Food_Apple_Pie + 104.9

In [28]:
#Carbohydrates' constraint
prob += lpSum([food_vars[x]*carbs[x] for x in food]) >= 130, "CarbsMinimum"
prob += lpSum([food_vars[x]*carbs[x] for x in food]) <= 200, "CarbsMaximum"

#Fat's constraint
prob += lpSum([food_vars[x]*fat[x] for x in food]) >= 20, "FatsMinimum"
prob += lpSum([food_vars[x]*fat[x] for x in food]) <= 50, "FatsMaximum"

#Protein's constraint
prob += lpSum([food_vars[x]*protein[x] for x in food]) >= 100, "ProteinsMinimum"
prob += lpSum([food_vars[x]*protein[x] for x in food]) <= 150, "ProteinsMaximum"

#Vit_A constraint
prob += lpSum([food_vars[x]*vit_A[x] for x in food]) >= 1000, "Vit_A_Minimum"
prob += lpSum([food_vars[x]*vit_A[x] for x in food]) <= 10000, "Vit_A_Maximum"

In [29]:
prob.solve()

1

In [30]:
prob.solver

In [31]:
LpStatus[prob.status]

'Optimal'

In [32]:
for var in prob.variables():
    print(f'Variable name: {var.name} , Variable value : {var.value()}\n')

print('\n')
print('*'*100)
print('\n')

#We can also see the slack variables of the constraints
for name, con in prob.constraints.items():
    print(f'constraint name:{name}, constraint value:{con.value()}\n')

print('*'*100)
print('\n')

## OBJECTIVE VALUE
print(f'OBJECTIVE VALUE IS: {round(prob.objective.value(),2)}')

Variable name: Food_Apple_Pie , Variable value : 0.0

Variable name: Food_Banana , Variable value : 0.0

Variable name: Food_Beef_Frankfurter , Variable value : 0.0

Variable name: Food_Chocolate_Chip_Cookies , Variable value : 0.0

Variable name: Food_Frozen_Broccoli , Variable value : 1.4079473

Variable name: Food_Frozen_Corn , Variable value : 0.0

Variable name: Food_Oatmeal_Cookies , Variable value : 0.0

Variable name: Food_Raw_Apple , Variable value : 0.0

Variable name: Food_Raw_Lettuce_Iceberg , Variable value : 0.0

Variable name: Food_Roasted_Chicken , Variable value : 1.2329408

Variable name: Food_Scrambled_Eggs , Variable value : 4.0165704

Variable name: Food_Spaghetti_W__Sauce , Variable value : 0.0

Variable name: Food_Tofu , Variable value : 0.0

Variable name: Food_Turkey_Bologna , Variable value : 0.0

Variable name: Food_Wheat_Bread , Variable value : 0.0

Variable name: Food_White_Bread , Variable value : 0.0

Variable name: Food__Baked_Potatoes , Variable value 

**Result:** 
The optimal value of the Diet is $5.58. Which includes the following food items: 

--> 2.64 servings of baked potatoes

--> 4.02 servings of scrambled eggs

--> 1.23 servings of Roasted chicken

--> 1.41 servings of Frozen broccoli

